In [2]:
%matplotlib ipympl
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pytransform3d.rotations import *

In [3]:
# load images and determine chessboard corners
left_img = cv2.imread("left.jpg", cv2.IMREAD_COLOR)
right_img = cv2.imread("right.jpg", cv2.IMREAD_COLOR)
left_img = cv2.resize(left_img, None, fx=0.25, fy=0.25)
right_img = cv2.resize(right_img, None, fx=0.25, fy=0.25)

In [4]:
patternsize = (9, 6)

def find_corners(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(img_gray, patternsize, flags=cv2.CALIB_CB_ADAPTIVE_THRESH+cv2.CALIB_CB_NORMALIZE_IMAGE+cv2.CALIB_CB_FAST_CHECK)
    #print(ret)
    if ret:
        corners = cv2.cornerSubPix(img_gray, corners, (11, 11), (-1, -1), criteria=(cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1))
    img = cv2.drawChessboardCorners(img, patternsize, corners, ret)
    return img, corners

left_img, corners_left = find_corners(left_img)
right_img, corners_right = find_corners(right_img)

In [5]:
# prepare images for intrinsic calibration
images = glob.glob('intrinsic/*.jpg')
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane
for image in images:
    img = cv2.imread(image, cv2.IMREAD_COLOR)
    img = cv2.resize(img, None, fx=0.25, fy=0.25)
    img, corners = find_corners(img)
    imgpoints.append(corners)
    
    # prepare object points
    objp = np.zeros((patternsize[1]*patternsize[0],3), np.float32)
    objp[:,:2] = np.mgrid[0:patternsize[0],0:patternsize[1]].T.reshape(-1,2)
    objpoints.append(objp)    

In [6]:
# calibrate camera intrinsics
retval, K, d, _, _ = cv2.calibrateCamera(objpoints, imgpoints, left_img.shape[:2][::-1], None, None)

In [7]:
# calibrate as stereo camera
retval, K1, d1, K2, d2, R, t, E, F = cv2.stereoCalibrate([objpoints[0]], [corners_left], [corners_right], K, d, K, d, left_img.shape[:2][::-1])

In [8]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
plot_basis(ax, np.eye(3), np.zeros(3,))
plot_basis(ax, R, t.reshape(3,))
plot_basis(ax, R.T, -np.matmul(R.T, t.reshape(3,)))
ax.set_xlim([-10,10])
ax.set_ylim([-10,10])
ax.set_zlim([-10,10])
plt.show()

FigureCanvasNbAgg()

In [9]:
pts1 = corners_left[:, 0, :]
pts2 = corners_right[:, 0, :]

In [10]:
# relative camera pose
R1 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]]).astype(np.float64)
t1 = np.array([[0], [0], [0]]).astype(np.float64)
R2 = R.T
t2 = -np.matmul(R.T, t.reshape(3,)).reshape(3,1)

In [11]:
## create projection matrices
proj_matrix1 = np.hstack([R1.T, -R1.T.dot(t1)])
proj_matrix2 = np.hstack([R2.T, -R2.T.dot(t2)])
proj_matrix1 = K.dot(proj_matrix1)
proj_matrix2 = K.dot(proj_matrix2)

In [12]:
pts1[:10]

array([[211.62093, 122.89432],
       [312.4482 , 124.71296],
       [408.48947, 127.01697],
       [500.55267, 129.6443 ],
       [587.4343 , 132.16008],
       [669.5428 , 134.70001],
       [747.61664, 137.55559],
       [821.4467 , 140.0841 ],
       [892.38666, 141.99115],
       [216.14915, 222.67738]], dtype=float32)

In [13]:
# undistort points prior to triangulation
pts1_undistorted = cv2.undistortPoints(pts1.reshape(-1, 1, 2), K, d, None, K).reshape(-1, 2)
pts2_undistorted = cv2.undistortPoints(pts2.reshape(-1, 1, 2), K, d, None, K).reshape(-1, 2)
pts = cv2.triangulatePoints(proj_matrix1, proj_matrix2, pts1_undistorted.T, pts2_undistorted.T).T
#pts = cv2.triangulatePoints(proj_matrix1, proj_matrix2, pts1.T, pts2.T).T

In [22]:
pts = cv2.convertPointsFromHomogeneous(pts).reshape(-1, 3)

In [23]:
pts

array([[-2.3588324 , -2.9734688 ,  8.760485  ],
       [-1.3528311 , -3.007264  ,  8.96096   ],
       [-0.36004767, -3.041054  ,  9.143721  ],
       [ 0.62875146, -3.0760314 ,  9.316667  ],
       [ 1.6030538 , -3.1154864 ,  9.496848  ],
       [ 2.5711918 , -3.1585531 ,  9.679421  ],
       [ 3.547523  , -3.2076178 ,  9.871115  ],
       [ 4.525416  , -3.262629  , 10.071691  ],
       [ 5.514843  , -3.3283677 , 10.32271   ],
       [-2.335406  , -1.979252  ,  8.8876505 ],
       [-1.3274802 , -2.0193815 ,  9.077079  ],
       [-0.32822645, -2.053373  ,  9.238956  ],
       [ 0.65543586, -2.0945582 ,  9.428022  ],
       [ 1.6274697 , -2.1371002 ,  9.621111  ],
       [ 2.5961926 , -2.1833918 ,  9.801267  ],
       [ 3.57245   , -2.2321956 ,  9.983808  ],
       [ 4.5505934 , -2.2874625 , 10.182944  ],
       [ 5.54368   , -2.3452234 , 10.4024    ],
       [-2.318017  , -0.98285836,  9.0119    ],
       [-1.2965109 , -1.0296584 ,  9.174084  ],
       [-0.29378313, -1.0716931 ,  9.333

In [24]:
# given:
#   - pts: 3D world points
#   - pts2_undistored: undistorted view of the world points in camera C2
# results:
#   - R, t to map 3D points to camera C2, we need to compute R.T, -R.T*t to retrieve the pose of C2 in world coordinates

retval, rvec, tvec, inliers = cv2.solvePnPRansac(pts.reshape(-1, 1, 3), pts2.reshape(-1, 1, 2), K, d, reprojectionError=8, iterationsCount=100)#, flags=cv2.SOLVEPNP_EPNP)
#retval, rvec, tvec = cv2.solveP3P(pts.reshape(-1, 1, 3)[:3, :, :], pts2_undistorted.reshape(-1, 1, 2)[:3, :, :], K, None, flags=cv2.SOLVEPNP_P3P)
#rvec = rvec[0]
#tvec = tvec[0]
print(retval)

# convert t vector from camera coords to world coords
R_recovered = cv2.Rodrigues(rvec)[0].T
t_recovered = -np.matmul(cv2.Rodrigues(rvec)[0].T, tvec)

print(R_recovered, t_recovered)

True
[[ 0.91591607  0.08015953 -0.39328388]
 [-0.08066949  0.99662405  0.01526233]
 [ 0.39317959  0.017747    0.91929041]] [[ 5.45086946]
 [-0.24347585]
 [ 1.24470032]]


In [25]:
R2-R_recovered

array([[ 0.00276028, -0.00149861,  0.0061841 ],
       [ 0.00142936,  0.00012618, -0.00077189],
       [-0.00619797, -0.00038519,  0.00263355]])

In [26]:
t2-t_recovered

array([[-0.05513268],
       [ 0.00871203],
       [-0.04619691]])

In [27]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2])
plot_basis(ax, R1, t1.reshape(3,))
plot_basis(ax, R2, t2.reshape(3,))
plot_basis(ax, R_recovered, t_recovered.reshape(3,))
ax.set_xlim([-10,10])
ax.set_ylim([-10,10])
ax.set_zlim([-10,10])
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
plt.show()

FigureCanvasNbAgg()

In [28]:
# map a world point to a camera point for testing
for pt in pts:
    test_pts = cv2.convertPointsToHomogeneous(pt.reshape(1, 3))
    R_recovered = cv2.Rodrigues(rvec)[0]
    t_recovered = tvec
    proj_matric_recovered = np.zeros((3, 4))
    proj_matric_recovered[:, :3] = R_recovered
    proj_matric_recovered[:, -1] = t_recovered.reshape(3,)
    proj_matric_recovered = np.matmul(K, proj_matric_recovered)
    img_pts = np.matmul(proj_matric_recovered, test_pts[0, 0, :]) 
    img_pts = cv2.convertPointsFromHomogeneous(img_pts.reshape(1, 3))
    print("world", pt, "reprojected image", img_pts)

world [-2.3588324 -2.9734688  8.760485 ] reprojected image [[[ 98.37130053 135.79725302]]]
world [-1.3528311 -3.007264   8.96096  ] reprojected image [[[179.5710102  134.19668607]]]
world [-0.36004767 -3.041054    9.143721  ] reprojected image [[[262.45248347 132.06975572]]]
world [ 0.62875146 -3.0760314   9.316667  ] reprojected image [[[348.55311829 129.45369559]]]
world [ 1.6030538 -3.1154864  9.496848 ] reprojected image [[[438.08961503 126.57091941]]]
world [ 2.5711918 -3.1585531  9.679421 ] reprojected image [[[531.57065128 123.30006984]]]
world [ 3.547523  -3.2076178  9.871115 ] reprojected image [[[630.72278916 119.52366832]]]
world [ 4.525416 -3.262629 10.071691] reprojected image [[[735.07523815 115.24644456]]]
world [ 5.514843  -3.3283677 10.32271  ] reprojected image [[[845.75147378 111.36801946]]]
world [-2.335406  -1.979252   8.8876505] reprojected image [[[101.82738375 224.83480287]]]
world [-1.3274802 -2.0193815  9.077079 ] reprojected image [[[181.61606535 224.39687807

In [29]:
rvec2, _ = cv2.Rodrigues(R2.T)
p2, _ = cv2.projectPoints(pts, rvec2, -t2, K, distCoeffs=d)
mse2 = np.linalg.norm(p2-pts2.reshape(-1, 1, 2))
print(p2[:10])
print(mse2)

[[[  11.308057  109.48553 ]]

 [[ 111.901344  106.63015 ]]

 [[ 214.93959   101.320244]]

 [[ 325.12274    95.81196 ]]

 [[ 442.25693    90.87912 ]]

 [[ 565.8122     86.42345 ]]

 [[ 696.351      82.752174]]

 [[ 833.57587    77.79324 ]]

 [[1006.23987    55.34331 ]]

 [[  17.505978  222.61194 ]]]
686.7485


In [30]:
rvec1, _ = cv2.Rodrigues(R1.T)
p1, _ = cv2.projectPoints(pts, rvec1, -t1, K, distCoeffs=d)
mse1 = np.linalg.norm(p1-pts1.reshape(-1, 1, 2))
print(p1[:10])
print(mse1)

[[[211.6582   122.06578 ]]

 [[312.46432  124.27023 ]]

 [[408.50275  126.585396]]

 [[500.57413  128.84879 ]]

 [[587.4546   131.23679 ]]

 [[669.55634  133.79903 ]]

 [[747.62836  136.49435 ]]

 [[821.47375  138.73965 ]]

 [[892.5044   139.88567 ]]

 [[216.15575  222.0639  ]]]
6.672421
